In [ ]:
from plateo.parsers import plate_from_platemap_spreadsheet
from bandwitch import BandsObservation, ClonesObservations, Clone, load_record
import flametree

In [ ]:
# PARAMETERS
data_dir = flametree.file_tree('.').directory_name  # rename `directory_name` to the dir containing the data
enzymes = ("NheI", "PstI")  # use a tuple
min_rfu_size_ratio = 0.3  # RFU / Size (bp) cutoff for each band
# Also set directory names below

In [ ]:
constructs_dict = {
    f._name_no_extension: load_record(
        f._path, topology='circular', id=f._name_no_extension)
    for f in data_dir.all_construct_records._all_files
}
# rename `all_construct_records` to the directory within `directory_name` which has the construct sequence files

In [ ]:
clones_plate = plate_from_platemap_spreadsheet(data_dir.name_of_plate_xlsx._path,
                                               data_field='construct')
# where `name_of_plate` is the name of the xlsx file that contains the plate layout
# note that filenames starting with a number may not be read properly

In [ ]:
clones_map = {
    well.name: well.data.construct
    for well in clones_plate.iter_wells()
    if 'construct' in well.data
    and str(well.data.construct) != 'nan'
}

bands_observations = BandsObservation.from_aati_fa_archive(
    data_dir.aati_files_zip._path,
    min_rfu_size_ratio=min_rfu_size_ratio)
# where `aati_files` is a zip of the fragment analyzer output

In [ ]:
clones = {
    well_name: Clone(name=well_name, construct_id=clones_map[well_name],
                     digestions={enzymes: bands_observations[well_name]})
    for well_name, construct_id in clones_map.items()
}

In [ ]:
# VALIDATE ALL CLONES WITH BANDWITCH
clones_observations = ClonesObservations(clones, constructs_dict)
validations = clones_observations.validate_all_clones(relative_tolerance=0.03)
validations_summary = clones_observations.validations_summary(validations)

In [ ]:
# CREATE A FOLDER WITH VALIDATION REPORTS
report_root = flametree.file_tree('.').results._dir('band_validation')
report_root._file('validations.pdf').write(
    clones_observations.plot_all_validations_patterns(validations)
)
ax = clones_observations.plot_validations_plate_map(validations)
ax.figure.savefig(report_root._file('success_map.pdf').open('wb'),
                  format='pdf', bbox_inches='tight')
# where `results` is the output folder name